In [ ]:
# Install necessary libraries if not already installed
# !pip install transformers torch sentencepiece

from transformers import T5ForConditionalGeneration, T5Tokenizer
import json
import os

# --- Setup: Load the fine-tuned model and tokenizer ---
# This assumes the model was saved to the './fine_tuned_t5_grammar_correction' directory.
model_path = "./fine_tuned_t5_grammar_correction"
try:
    model = T5ForConditionalGeneration.from_pretrained(model_path)
    tokenizer = T5Tokenizer.from_pretrained(model_path)
    print("Fine-tuned model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}. Please ensure you have fine-tuned a model first.")
    # Fallback to a base model for demonstration if fine-tuned model isn't available
    print("Using a base T5 model for demonstration purposes.")
    model_name = "t5-small"
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)


# --- Core Proofreading Logic ---
prefix = "grammar correction: "

def get_correction(text):
    """Uses the loaded T5 model to generate a correction for the given text."""
    input_text = prefix + text
    inputs = tokenizer(input_text, return_tensors="pt")
    
    # Generate the corrected output
    outputs = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=128
    )
    
    # Decode and return the corrected text
    corrected_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected_text

# --- User Feedback Loop Simulation ---
feedback_file = "user_feedback.jsonl"

def collect_feedback(original, corrected, helpfulness):
    """Simulates collecting user feedback and saving it to a file."""
    feedback_entry = {
        "original_text": original,
        "model_correction": corrected,
        "was_helpful": helpfulness
    }
    
    # Append the feedback entry to a JSONL file
    with open(feedback_file, "a") as f:
        f.write(json.dumps(feedback_entry) + "\n")
    print("Feedback recorded.")

# --- Application Flow ---
def proofreading_app():
    """Main function to run the proofreading application."""
    print("--- Proofreading Application ---")
    print("Type 'exit' to quit.")

    while True:
        user_input = input("\nEnter your text: ")
        if user_input.lower() == "exit":
            print("Thank you for using the proofreading application. Goodbye!")
            break

        # Get correction from the model
        corrected_text = get_correction(user_input)

        # Display results to the user
        print(f"\nOriginal Text:   '{user_input}'")
        print(f"Corrected Text:  '{corrected_text}'")

        # Simulate the user feedback loop
        while True:
            feedback = input("Was this correction helpful? (y/n): ").lower()
            if feedback in ["y", "n"]:
                is_helpful = True if feedback == "y" else False
                collect_feedback(user_input, corrected_text, is_helpful)
                break
            else:
                print("Invalid input. Please enter 'y' or 'n'.")

if __name__ == "__main__":
    proofreading_app()


